In [1]:
import nibabel as nib
import numpy as np
import os
import glob
from tqdm import tqdm

In [2]:
# Get a list of all nifti images in the data folder
brains=glob.glob('val/**/*.nii')

## Check affine on all files

In [3]:
# loop over all images
for brain in tqdm(brains):
    img = nib.load(brain)
    # get affine of the image
    affine = img.affine
    # ignore values smaller than 1E-6, by replacing them with 0
    affine[abs(affine)<1E-6]=0
    # Check if the affine uses a combination of the 3 axes
    # by countint the non-zero element in the 3x3 part of the affine matrix
    if np.count_nonzero(affine[:3,:3])!=3:
        print("Strange combination of axes in affine for image_file: "+brain)
    

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 1311.47it/s]


## Code to addapt the origin !?

In [4]:
for brain in tqdm(brains):
    img = nib.load(brain)
    
    # get affine of the image
    affine = img.affine
    
    # ignore values smaller than 1E-6, by replacing them with 0
    # copy affine to a working copy
    affine_to_work=affine
    affine_to_work[abs(affine)<1E-6]=0
    
    # Get the voxel size
    vox=np.sqrt(np.sum(affine_to_work[:3,:3]**2))
    
    # Get the image header
    header=img.header
    # Get dimensions of the image
    shape=np.array(header.get_data_shape())
    
    # Create offset to the centre of the image
    off = np.sum(affine_to_work[:3,:3],axis=1)*(shape/2)*-1
    
    # Set the new offset on the original affine
    affine[:3,3]=off
    # Set the new affine with the updated origin 
    img.set_sform(affine)
    
    # Save image file with new affine
    img.to_filename(brain.replace('brainmask.nii','brainmask_newOrig.nii'))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:02<00:00, 82.80it/s]
